<a href="https://colab.research.google.com/github/CliffBooth/telecom_labs/blob/main/code/lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Get thinkdsp.py
import os
if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py
from thinkdsp import *
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
InteractiveShell.ast_node_interactivity = 'all'

# **Лабараторная работа №7**

**Exercise #7.2**

In this chapter, I showed how we can express the DFT and
inverse DFT as matrix multiplications. These operations take time propor-
tional to N 2, where N is the length of the wave array. That is fast enough for
many applications, but there is a faster algorithm, the Fast Fourier Transform
(FFT), which takes time proportional to N log N .

the key to the FFT is the Danielson-Lanczos lemma:
`DFT(y)[n] = DFT(e)[n] + exp(−2πin/N )DFT(o)[n]`
Where DF T (y)[n] is the nth element of the DFT of y; e is a wave array
containing the even elements of y, and o contains the odd elements of y.
This lemma suggests a recursive algorithm for the DFT:
1. Given a wave array, y, split it into its even elements, e, and its odd
elements, o.
2. Compute the DFT of e and o by making recursive calls.
3. Compute DFT(y) for each value of n using the Danielson-Lanczos
lemma.

For the base case of this recursion, you could wait until the length of y is
1 . In that case, DFT(y) = y. Or if the length of y is sufficiently small, you
could compute its DFT by matrix multiplication, possibly using a precomputed
matrix.

Нужно реализовать алгоритм БПФ, упомянутая Danielson-Lanczos lemma предпологает рукурсивную реализацию алгоритма.

Библиотечный алгоритм fft:


In [22]:
ys = [0.4, 0.2, 0.5, -0.3]
hs = np.fft.fft(ys)
print(hs)

[ 0.8+0.j  -0.1-0.5j  1. +0.j  -0.1+0.5j]


Реализация DFT из учебника

In [23]:
def dft(ys):
  N = len(ys)
  ts = np.arange(N) / N
  freqs = np.arange(N)
  args = np.outer(ts, freqs)
  M = np.exp(1j * PI2 * args)
  amps = M.conj().transpose().dot(ys)
  return amps

Увидим, что получаем такой же результат, что и от библиотечной функции (не считая погрешность)

In [24]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.777195988230628e-16

Напишем нерекурсивную функцию, которая разделит входной массив на подмассивы четных и нечетных элементов и использует библиотечную функцию для расчета БПФ подмассивов.


In [25]:
def fft_norec(ys):
  N = len(ys)
  He = np.fft.fft(ys[::2])
  Ho = np.fft.fft(ys[1::2])
  ns = np.arange(N)
  W = np.exp(-1j * PI2 * ns / N)
  return np.tile(He, 2) + W * np.tile(Ho, 2)

Результат, как видно, остается таким же

In [26]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

1.3714655826180364e-16

Теперь, напишем рекурсивное решение:

In [29]:
def fft(ys):
  N = len(ys)
  if N == 1:
    return ys
  He = fft(ys[::2])
  Ho = fft(ys[1::2])
  ns = np.arange(N)
  W = np.exp(-1j * PI2 * ns / N)
  return np.tile(He, 2) + W * np.tile(Ho, 2)

Убедимся, что получаем такой же результат:

In [30]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

2.7984961504774455e-16